In [11]:
!pip -q install langchain-groq

In [12]:
import os
import re
import csv
import random
from tqdm import tqdm
import pandas as pd
from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
sample_data = pd.read_csv('../train_database/sample.csv')
sample_data_fulltext = [sample_data["text"][9072], sample_data["text"][68], sample_data["text"][3]]
sample_data_summary = [sample_data["summary"][9072], sample_data["summary"][68], sample_data["summary"][3]]
navernews_data = pd.read_csv('../../Non_Finance_data/Naver_Stock/파일이름.csv')

email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
bracket_pattern = r"\[.*?\]"
reporter_pattern = r"\s*[가-힣]+\s*기자\b.*$"

Articles = []

for i in range(len(navernews_data)):
    title = str(navernews_data['Title'][i])
    body = str(navernews_data['Body'][i])

    if re.search(email_pattern, body):
        body = re.split(email_pattern, body)[0]
    
    body = re.sub(bracket_pattern, '', body)
    body = re.sub(reporter_pattern, '', body, flags=re.DOTALL)
    
    Articles.append(f"{title}\n\n {body}")

duplicate_list = pd.read_csv('../train_database/Summary_gpt.csv')['id'].tolist()
dd = pd.read_csv('../train_database/Summary_groq.csv')['id'].tolist()
duplicate_list.extend(dd)

In [14]:
random.seed(12)
random_idx = random.sample(range(0, len(navernews_data)), 300)

pseudo_summary_dataset = list(filter(lambda x: len(x[1]) > 30 and x[0] not in duplicate_list, [[i, Articles[i]] for i in random_idx]))

sample = [f"Original Text: {sample_data_fulltext[i]} >> Summary Text: {sample_data_summary[i]}" for i in range(3)]
sample = "\n\n".join(sample)
print(len(pseudo_summary_dataset))

112


In [15]:
load_dotenv()

llm = ChatGroq(
    model='deepseek-r1-distill-llama-70b',
    temperature=0,
    max_tokens=None,
    timeout=None,
    api_key=os.getenv('GROQ_API_KEY')
)

In [16]:
system = 'You are a helpful assistant, specialized in Econmic analysis'
human = '{text}'
prompt_template = ChatPromptTemplate.from_messages([
        ('system', system),
        ('human', human)
    ])
chain = prompt_template | llm

In [17]:
def construct_CoT_prompt(origin_exemplars, summarized_exemplars):
    if len(origin_exemplars) != len(summarized_exemplars): return -1

    instruction = 'Instruction: Here is **GOOD** summaries of the news article data.\n'
    instruction += 'Generate the summary for the given summary example step-by-step rationale.\n'
    instruction += 'Provide your final summary based on the rationale using an identifier "####".'
    instruction += '요약은 반드시 한국어여야 하며, 두 문장으으로 요약을 해야 합니다.'
    prompt = instruction
    for i in range(len(origin_exemplars)):
        origin_exemplar = origin_exemplars[i]
        summarized_exemplar = summarized_exemplars[i]
        prompt += f'\n[Example {i+1}]\n'
        prompt += f'Original text:\n{origin_exemplar}\n'
        prompt += f'Summarized text:\n{summarized_exemplar}'

    prompt += f'\n[Example {len(origin_exemplar)+1}]'
    prompt += 'Original text:\n{original_text}\n'

    return prompt

construct_CoT_prompt(sample_data_fulltext, sample_data_summary)

'Instruction: Here is **GOOD** summaries of the news article data.\nGenerate the summary for the given summary example step-by-step rationale.\nProvide your final summary based on the rationale using an identifier "####".요약은 반드시 한국어여야 하며, 두 문장으으로 요약을 해야 합니다.\n[Example 1]\nOriginal text:\n올해 기업공개(IPO) \'최대어\'로 꼽히는 빅히트엔터테인먼트(이하 빅히트)의 공모주 청약에 관심이 쏠리고 있다.\n  예고편 격인 수요예측 흥행에 성공하면서 기대감이 커지는 분위기다.\n  지난 24~25일 진행한 기관 투자가 대상의 수요예측 경쟁률은 1117.\n 3대 1에 달했다.\n  카카오게임즈(1478.\n 5대 1)보다 낮지만, SK바이오팜(835.\n 7대 1)보다는 높은 수준이다.\n  국내외 기관 1420곳이 참여했다.\n   공모가 13만5000원…수요예측 1117대 1공모가는 주당 13만5000원으로 정해졌다.\n  당초 희망했던 공모액 10만5000~13만5000원의 최상단 가격이다.\n  수요예측에 참여한 기관의 97%에 달하는 1381곳이 13만5000원 이상을 써냈다.\n  빅히트 소속 아이돌 그룹인 방탄소년단(BTS)의 글로벌 인지도가 영향을 미쳤다는 평가다.\n  이에 따른 공모자금은 총 9625억5000만원, 시가총액은 4조8000억원 정도다.\n  국내 \'빅3\' 엔터사인 JYP와 YG, SM 시총 합계(약 3조2000억원)를 뛰어넘는 규모다.\n  2005년 설립한 빅히트는 BTS를 세계적인 그룹으로 키워낸 엔터테인먼트 회사다.\n  이 회사는 지난달 한국거래소에서 코스피 상장 예비심사 승인을 받았고, 지난 2일 증권신고서를 제출했다.\n  공모 주식 수는 총 713만주다.\n  이 중 일반 청약자 몫은 전체의 

In [18]:
CoT_prompt = construct_CoT_prompt(sample_data_fulltext, sample_data_summary)
Input_default = CoT_prompt.format(original_text=pseudo_summary_dataset[0][1])

In [ ]:
batch_results = []
fieldnames = ['id', 'summary']
csv_file = '../train_database/Summary_groq.csv'

for idx in tqdm(range(len(pseudo_summary_dataset))):
    cur_original_text = pseudo_summary_dataset[idx][1]
    cur_model_input = CoT_prompt.format(original_text=cur_original_text)
    
    response = chain.invoke({"text":cur_model_input})
    result = response.content
    if 'Error:' in result:
        print(f'Skip {i+1}th task')
        continue

    final_summary = result.split("####")[-1].strip()

    batch_results.append({'id': pseudo_summary_dataset[idx][0], 'summary': final_summary})

    if (idx + 1) % 3 == 0:
        file_exists = os.path.exists(csv_file)
        with open(csv_file, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            if not file_exists or os.path.getsize(csv_file) == 0:
                writer.writeheader()
            writer.writerows(batch_results)
        batch_results = []

if batch_results:
    file_exists = os.path.exists(csv_file)
    with open(csv_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists or os.path.getsize(csv_file) == 0:
            writer.writeheader()
        writer.writerows(batch_results)

  3%|▎         | 3/112 [00:56<34:28, 18.97s/it]


KeyboardInterrupt: 